In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [4]:
traindata=pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv")
testdata=pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv")
traindata.head(5)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [5]:
traindata.shape

(11504798, 12)

In [6]:
traindata.isna().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

In [7]:
def clean_data(data, fit=False):
    categorical_columns = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']
    data[categorical_columns] = data[categorical_columns].fillna(data[categorical_columns].mode().iloc[0])
    for col in categorical_columns:
        if fit:
            le = LabelEncoder()
            data[col] = le.fit_transform(data[col].astype(str))
            clean_data.encoders[col] = le
        else:
            le = clean_data.encoders[col]
            data[col] = data[col].astype(str).map(dict(zip(le.classes_, le.transform(le.classes_))))
            data[col] = data[col].fillna(-1).astype(int)

    return data

In [8]:
clean_data.encoders = {}

In [9]:
traindata = clean_data(traindata, fit=True)
testdata = clean_data(testdata, fit=False)

In [10]:
traindata

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,1,21,1,35.0,0,0,1,65101.0,124.0,187,0
1,1,1,43,1,28.0,0,2,1,58911.0,26.0,288,1
2,2,0,25,1,14.0,1,1,0,38043.0,152.0,254,0
3,3,0,35,1,1.0,0,0,1,2630.0,156.0,76,0
4,4,0,36,1,15.0,1,0,0,31951.0,152.0,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,11504793,1,48,1,6.0,0,0,1,27412.0,26.0,218,0
11504794,11504794,0,26,1,36.0,0,1,1,29509.0,152.0,115,1
11504795,11504795,0,29,1,32.0,1,1,0,2630.0,152.0,189,0
11504796,11504796,0,51,1,28.0,0,0,1,48443.0,26.0,274,1


In [11]:
X=traindata.drop(['Response','id'],axis=1)
y=traindata['Response']

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:

logiregmodel=LogisticRegression(max_iter=1000)
logiregmodel.fit(x_train,y_train)
logireg_y_pred=logiregmodel.predict(x_test)
val_accuracy = accuracy_score(y_test, logireg_y_pred)
print(f"Accuracy Score: {val_accuracy}") # Use accuracy_score for classification

# Optionally, you can also look at other classification metrics
from sklearn.metrics import precision_score, recall_score, f1_score
print(f"Precision: {precision_score(y_test, logireg_y_pred)}") 
print(f"Recall: {recall_score(y_test, logireg_y_pred)}") 
print(f"F1-Score: {f1_score(y_test, logireg_y_pred)}") 


Accuracy Score: 0.8750004346012099
Precision: 0.3223868642370845
Recall: 0.01421337363692864
F1-Score: 0.02722639167447382


In [14]:

DTCmodel=DecisionTreeClassifier()
DTCmodel.fit(x_train,y_train)
DTC_y_pred=DTCmodel.predict(x_test)
val_accuracy = accuracy_score(y_test, DTC_y_pred)
print(f"Accuracy Score: {val_accuracy}") # Use accuracy_score for classification
# Optionally, you can also look at other classification metrics
from sklearn.metrics import precision_score, recall_score, f1_score
print(f"Precision: {precision_score(y_test, DTC_y_pred)}") 
print(f"Recall: {recall_score(y_test, DTC_y_pred)}") 
print(f"F1-Score: {f1_score(y_test, DTC_y_pred)}") 


Accuracy Score: 0.8324799214241012
Precision: 0.327435688904037
Recall: 0.34263941465619524
F1-Score: 0.33486506866925386


In [15]:
XGBmodel = xgb.XGBClassifier()

# Fit the model
XGBmodel.fit(x_train, y_train)

# Predict using the model
XGB_y_pred = XGBmodel.predict(x_test)

# Calculate accuracy
val_accuracy = accuracy_score(y_test, XGB_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, XGB_y_pred)
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, XGB_y_pred)
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, XGB_y_pred)
print(f"F1-Score: {f1}")

Accuracy Score: 0.8799144704818859
Precision: 0.5778185441862572
Recall: 0.09009336685688457
F1-Score: 0.15588168804110739


In [21]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [22]:
from catboost import CatBoostClassifier

# Initialize the CatBoost model
CATmodel = CatBoostClassifier(verbose=0)  # Set verbose=0 to suppress output

# Fit the model
CATmodel.fit(x_train, y_train)

# Predict using the model
CAT_y_pred = CATmodel.predict(x_test)

# Calculate accuracy
val_accuracy = accuracy_score(y_test, CAT_y_pred)
print(f"Accuracy Score: {val_accuracy}")

# Calculate precision
precision = precision_score(y_test, CAT_y_pred)
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, CAT_y_pred)
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, CAT_y_pred)
print(f"F1-Score: {f1}")


Accuracy Score: 0.8811309192684792
Precision: 0.5728402500564886
Recall: 0.13428724786711113
F1-Score: 0.2175707800176789


In [23]:
test_predictions = CATmodel.predict(testdata.drop(['id'], axis=1))
# Create submission file
submission = pd.DataFrame({'id': testdata['id'], 'Response': test_predictions})
submission.to_csv('submission.csv', index=False)
print("\nSubmission file created successfully!")


Submission file created successfully!
